In [183]:
import sys
import os
import importlib
import re
import math
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(os.path.join(project_dir, 'src', 'utilities'))
import utilities as utils

In [184]:
# Import if using in a the notebook within a browser
# from IPython.core.display import HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

In [185]:
# Run if changes are made to the datasets_config file or utilities
importlib.reload(utils)

<module 'utilities' from 'c:\\Users\\Jonah\\Development\\binary-rainfall-time-series-ml\\src\\utilities\\utilities.py'>

In [186]:
io_dir = os.path.join(project_dir, 'data')
raw_dir = os.path.join(io_dir, 'raw')
processed_dir = os.path.join(io_dir, 'processed')

In [187]:
raw_train_df = pd.read_csv(os.path.join(raw_dir, 'train.csv'), index_col=0)
raw_test_df = pd.read_csv(os.path.join(raw_dir, 'test.csv'), index_col=0)
raw_sample_submission_df = pd.read_csv(os.path.join(raw_dir, 'sample_submission.csv'), index_col=0)

In [188]:
raw_train_df.shape
raw_test_df.shape
raw_sample_submission_df.shape
raw_train_df.head()
raw_test_df.head()
raw_sample_submission_df.head()

(2190, 12)

(730, 11)

(730, 1)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


,rainfall
id,
2190,0
2191,0
2192,0
2193,0
2194,0


### Let's check if the days are consecutive and fall within 365 years, and if any leap years are present.

In [189]:
combined_df = pd.concat(
    [raw_train_df, raw_test_df.merge(raw_sample_submission_df, left_index=True, right_index=True)],
    axis=0)
combined_df['source_data'] = combined_df.apply(lambda row: 'Train' if row.name in raw_train_df.index else 'Test', axis=1)
combined_df.index.is_unique
raw_train_df.shape
raw_test_df.shape
raw_sample_submission_df.shape
combined_df.loc[combined_df['source_data'] == 'Train'].shape
combined_df.loc[combined_df['source_data'] == 'Test'].shape
combined_df.head()
combined_df.tail()

True

(2190, 12)

(730, 11)

(730, 1)

(2190, 13)

(730, 13)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data
id,,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1,Train
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1,Train
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1,Train
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1,Train
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0,Train


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data
id,,,,,,,,,,,,,
2915,361,1020.8,18.2,17.6,16.1,13.7,96.0,95.0,0.0,20.0,34.3,0,Test
2916,362,1011.7,23.2,18.1,16.0,16.0,78.0,80.0,1.6,40.0,25.2,0,Test
2917,363,1022.7,21.0,18.5,17.0,15.5,92.0,96.0,0.0,50.0,21.9,0,Test
2918,364,1014.4,21.0,20.0,19.7,19.8,94.0,93.0,0.0,50.0,39.5,0,Test
2919,365,1020.9,22.2,18.8,17.0,13.3,79.0,89.0,0.2,60.0,50.6,0,Test


In [190]:
combined_df.groupby(['day'])[['rainfall']].count()
combined_df.groupby(['day'])[['rainfall']].count().max()
combined_df.groupby(['day'])[['rainfall']].count().min()

,rainfall
day,
1,8
2,8
3,9
4,9
5,9
...,...
361,8
362,8
363,7


rainfall    9
dtype: int64

rainfall    7
dtype: int64

In [191]:
# add 1 since the index starts at 0
combined_df['expected_day'] = (combined_df.index) % 365 + 1
combined_df['mislabeled_day'] = combined_df.apply(lambda x: True if x['day'] != x['expected_day'] else False, axis=1)
combined_df['elapsed_days'] = np.arange(len(combined_df.index))
# add 1 since elasped_days starts at 0
combined_df['year'] = combined_df['elapsed_days'].apply(lambda x: math.ceil((x + 1) / 365))

combined_df.loc[combined_df['source_data'] == 'Train']['year'].unique()
combined_df.loc[combined_df['source_data'] == 'Train']['mislabeled_day'].unique()
combined_df.loc[combined_df['source_data'] == 'Test']['year'].unique()
combined_df.loc[combined_df['source_data'] == 'Test']['mislabeled_day'].unique()

array([1, 2, 3, 4, 5, 6])

array([False,  True])

array([7, 8])

array([False])

In [192]:
non_consecutive_day_indexes = combined_df[combined_df['mislabeled_day']].index.to_list()
window_distance = 2
potential_corrections = []
original_and_corrected_days = {}
for i in non_consecutive_day_indexes:
    window = [j for j in range(i-window_distance, i+window_distance+1)]
    combined_df.reindex(window)
    day = combined_df.at[i, 'day']
    expected_day = combined_df.at[i, 'expected_day']
    potential_corrections.append(expected_day)
    if day not in original_and_corrected_days:
        original_and_corrected_days[day] = [expected_day]
    else:
        original_and_corrected_days[day].append(expected_day)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1035,306,1016.9,27.5,24.5,23.5,22.4,82.0,88.0,4.5,50.0,24.9,1,Train,306,False,1035,3
1036,307,1014.6,25.2,24.8,23.9,21.2,84.0,70.0,2.1,80.0,33.5,1,Train,307,False,1036,3
1037,3,1015.2,25.0,23.9,23.6,19.9,76.0,61.0,8.8,90.0,14.1,0,Train,308,True,1037,3
1038,309,1017.0,27.7,24.0,22.6,19.6,91.0,78.0,3.3,25.0,16.4,1,Train,309,False,1038,3
1039,310,1018.4,26.4,24.4,23.3,23.3,86.0,66.0,9.7,60.0,8.3,1,Train,310,False,1039,3


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1130,36,1017.0,22.4,20.2,18.5,13.1,76.0,69.0,7.6,40.0,11.3,0,Train,36,False,1130,4
1131,37,1017.5,23.1,19.9,17.4,16.8,80.0,88.0,0.0,50.0,39.5,1,Train,37,False,1131,4
1132,265,1020.6,18.4,17.1,16.9,15.0,75.0,86.0,0.0,40.0,24.9,1,Train,38,True,1132,4
1133,39,1014.9,23.3,21.6,20.9,20.9,87.0,88.0,0.0,60.0,15.1,1,Train,39,False,1133,4
1134,40,1014.0,23.0,21.1,20.6,19.1,80.0,87.0,0.1,70.0,14.5,1,Train,40,False,1134,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1208,114,1013.0,28.8,25.6,23.6,21.6,91.0,90.0,1.3,80.0,24.9,0,Train,114,False,1208,4
1209,115,1012.3,30.8,27.4,26.8,24.5,84.0,81.0,2.3,100.0,14.3,1,Train,115,False,1209,4
1210,80,1011.5,29.8,26.8,25.1,23.2,87.0,81.0,2.5,20.0,12.7,1,Train,116,True,1210,4
1211,117,1011.6,28.4,26.1,24.8,25.3,85.0,83.0,3.7,60.0,14.3,1,Train,117,False,1211,4
1212,118,1013.0,28.6,25.5,24.8,22.2,76.0,74.0,6.0,60.0,9.2,1,Train,118,False,1212,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1249,155,1008.2,34.0,30.2,28.2,26.5,81.0,79.0,6.8,290.0,43.8,1,Train,155,False,1249,4
1250,156,1008.1,34.0,30.8,28.7,25.2,74.0,22.0,11.8,240.0,12.5,0,Train,156,False,1250,4
1251,236,1005.0,35.3,31.0,26.4,26.4,74.0,49.0,10.8,230.0,28.6,1,Train,157,True,1251,4
1252,158,1008.9,33.4,30.4,28.6,25.7,82.0,72.0,7.7,60.0,40.8,0,Train,158,False,1252,4
1253,159,1005.3,32.0,29.2,26.2,26.3,87.0,85.0,1.5,220.0,4.5,1,Train,159,False,1253,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1282,188,1008.3,33.2,30.0,28.8,25.2,81.0,77.0,7.8,220.0,22.5,1,Train,188,False,1282,4
1283,189,1009.5,33.3,30.0,27.8,25.8,79.0,43.0,9.5,220.0,11.2,0,Train,189,False,1283,4
1284,4,1007.9,33.0,30.1,27.8,25.1,78.0,67.0,5.7,220.0,12.5,1,Train,190,True,1284,4
1285,191,1005.7,31.0,29.2,27.8,26.2,83.0,83.0,4.7,60.0,14.8,1,Train,191,False,1285,4
1286,192,1008.0,33.0,30.2,28.4,25.2,78.0,69.0,10.8,220.0,12.0,0,Train,192,False,1286,4


...

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1450,356,1016.8,21.1,18.8,17.4,15.3,75.0,88.0,0.0,50.0,30.3,1,Train,356,False,1450,4
1451,357,1022.2,21.6,18.2,16.4,14.6,77.0,88.0,0.0,50.0,24.8,1,Train,357,False,1451,4
1452,140,1023.2,20.6,17.8,14.4,13.4,75.0,88.0,0.0,20.0,23.3,1,Train,358,True,1452,4
1453,82,1017.1,18.9,17.5,16.5,16.8,88.0,91.0,0.0,20.0,30.0,1,Train,359,True,1453,4
1454,360,1016.8,18.9,17.7,16.9,16.8,96.0,95.0,0.0,50.0,17.5,1,Train,360,False,1454,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1451,357,1022.2,21.6,18.2,16.4,14.6,77.0,88.0,0.0,50.0,24.8,1,Train,357,False,1451,4
1452,140,1023.2,20.6,17.8,14.4,13.4,75.0,88.0,0.0,20.0,23.3,1,Train,358,True,1452,4
1453,82,1017.1,18.9,17.5,16.5,16.8,88.0,91.0,0.0,20.0,30.0,1,Train,359,True,1453,4
1454,360,1016.8,18.9,17.7,16.9,16.8,96.0,95.0,0.0,50.0,17.5,1,Train,360,False,1454,4
1455,361,1016.4,16.3,16.6,15.9,15.0,91.0,96.0,0.0,70.0,37.0,1,Train,361,False,1455,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1455,361,1016.4,16.3,16.6,15.9,15.0,91.0,96.0,0.0,70.0,37.0,1,Train,361,False,1455,4
1456,362,1013.9,18.9,17.1,15.3,16.9,79.0,94.0,0.0,70.0,24.0,1,Train,362,False,1456,4
1457,144,1013.0,22.2,20.5,19.1,17.9,79.0,81.0,2.2,70.0,28.6,1,Train,363,True,1457,4
1458,17,1014.5,18.7,17.6,17.0,17.4,97.0,95.0,0.0,50.0,25.0,1,Train,364,True,1458,4
1459,78,1016.6,21.1,19.1,17.1,13.0,94.0,97.0,0.0,30.0,50.6,1,Train,365,True,1459,4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1456,362,1013.9,18.9,17.1,15.3,16.9,79.0,94.0,0.0,70.0,24.0,1,Train,362,False,1456,4
1457,144,1013.0,22.2,20.5,19.1,17.9,79.0,81.0,2.2,70.0,28.6,1,Train,363,True,1457,4
1458,17,1014.5,18.7,17.6,17.0,17.4,97.0,95.0,0.0,50.0,25.0,1,Train,364,True,1458,4
1459,78,1016.6,21.1,19.1,17.1,13.0,94.0,97.0,0.0,30.0,50.6,1,Train,365,True,1459,4
1460,1,1021.9,19.3,18.1,17.3,16.3,95.0,100.0,0.0,70.0,26.5,1,Train,1,False,1460,5


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall,source_data,expected_day,mislabeled_day,elapsed_days,year
id,,,,,,,,,,,,,,,,,
1457,144,1013.0,22.2,20.5,19.1,17.9,79.0,81.0,2.2,70.0,28.6,1,Train,363,True,1457,4
1458,17,1014.5,18.7,17.6,17.0,17.4,97.0,95.0,0.0,50.0,25.0,1,Train,364,True,1458,4
1459,78,1016.6,21.1,19.1,17.1,13.0,94.0,97.0,0.0,30.0,50.6,1,Train,365,True,1459,4
1460,1,1021.9,19.3,18.1,17.3,16.3,95.0,100.0,0.0,70.0,26.5,1,Train,1,False,1460,5
1461,2,1015.3,21.9,18.7,17.3,16.8,85.0,95.0,0.0,60.0,21.9,1,Train,2,False,1461,5


### Potentially mislabeled days
 * There is only one row in which the 'expected_day' is in the set of the 'day' values, so the rows with a non consecutive 'day' are not simply in the wrong location. 
 * When looking at 'temperature' and 'dewpoint' graphically, the trend is better maintained when 'expected_day' is used instead of 'day' when 'expected_day' and 'day' do not match.
 * We will use 'expected_day' during training and prediction.

In [193]:
for pc in potential_corrections:
    if pc in original_and_corrected_days:
        f'{pc} {original_and_corrected_days[pc]}'
set(original_and_corrected_days.keys()).intersection(potential_corrections)

'116 [np.int64(334)]'

{np.int64(116)}

In [194]:
fig = px.scatter(combined_df, x='day', y='temparature', symbol='source_data', color='mislabeled_day', facet_col='year')
fig.show()
fig = px.scatter(combined_df, x='expected_day', y='temparature', symbol='source_data', color='mislabeled_day', facet_col='year')
fig.show()
fig = px.scatter(combined_df, x='elapsed_days', y='temparature', symbol='source_data', color='mislabeled_day')
fig.show()

In [195]:
fig = px.scatter(combined_df, x='day', y='dewpoint', symbol='source_data', color='mislabeled_day', facet_col='year')
fig.show()
fig = px.scatter(combined_df, x='expected_day', y='dewpoint', symbol='source_data', color='mislabeled_day', facet_col='year')
fig.show()
fig = px.scatter(combined_df, x='elapsed_days', y='dewpoint', symbol='source_data', color='mislabeled_day')
fig.show()

## Train data

In [196]:
raw_train_df.head()
raw_train_df.info()
raw_train_df.describe()
# Check for duplicate feature names
raw_train_df.loc[:, raw_train_df.columns.duplicated()].shape
# Check for missing values 
raw_train_df.isna().sum()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


<class 'pandas.core.frame.DataFrame'>
Index: 2190 entries, 0 to 2189
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2190 non-null   int64  
 1   pressure       2190 non-null   float64
 2   maxtemp        2190 non-null   float64
 3   temparature    2190 non-null   float64
 4   mintemp        2190 non-null   float64
 5   dewpoint       2190 non-null   float64
 6   humidity       2190 non-null   float64
 7   cloud          2190 non-null   float64
 8   sunshine       2190 non-null   float64
 9   winddirection  2190 non-null   float64
 10  windspeed      2190 non-null   float64
 11  rainfall       2190 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 287.0 KB


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
count,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000,2190.000000
mean,179.948402,1013.602146,26.365799,23.953059,22.170091,20.454566,82.036530,75.721918,3.744429,104.863151,21.804703,0.753425
std,105.203592,5.655366,5.654330,5.222410,5.059120,5.288406,7.800654,18.026498,3.626327,80.002416,9.898659,0.431116
min,1.000000,999.000000,10.400000,7.400000,4.000000,-0.300000,39.000000,2.000000,0.000000,10.000000,4.400000,0.000000
25%,89.000000,1008.600000,21.300000,19.300000,17.700000,16.800000,77.000000,69.000000,0.400000,40.000000,14.125000,1.000000
50%,178.500000,1013.000000,27.800000,25.500000,23.850000,22.150000,82.000000,83.000000,2.400000,70.000000,20.500000,1.000000
75%,270.000000,1017.775000,31.200000,28.400000,26.400000,25.000000,88.000000,88.000000,6.800000,200.000000,27.900000,1.000000
max,365.000000,1034.600000,36.000000,31.500000,29.800000,26.700000,98.000000,100.000000,12.100000,300.000000,59.500000,1.000000


(2190, 0)

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
rainfall         0
dtype: int64

### Basic cleanup
 * Make sure the index has only unique values
 * Remove any leading and trailing whitespace in the column names
 * Check for duplcated rows and columns
 * Check for:
     * rows and columns that contain all null/missing values
     * features (columns) that are invariable

In [197]:
train_df = raw_train_df.copy()
train_df.index.is_unique
# Remove leading and trailing whitespace form the column names
train_df.rename(columns=lambda x: x.strip(), inplace=True)
train_df.head()
train_df.tail()

True

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
2185,361,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,22.1,1
2186,362,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,35.3,1
2187,363,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,32.9,1
2188,364,1022.3,16.4,15.2,13.8,14.7,92.0,93.0,0.1,40.0,18.0,1
2189,365,1013.8,21.2,19.1,18.0,18.0,89.0,88.0,1.0,70.0,48.0,1


In [198]:
train_df.shape
train_df.dropna(how='all').dropna(how='all', axis=1).shape
train_df.duplicated().any()
train_df.columns.duplicated().any()
for col in train_df:
    # The column is invariable if this is true
    if train_df[col].nunique() == 1:
        col

(2190, 12)

(2190, 12)

np.False_

np.False_

## Test data
#### The same data cleaning and transorming done to the train data will be done to the test data

In [199]:
raw_test_df.head()
raw_test_df.info()
raw_test_df.describe()
# Check for duplicate feature names
raw_test_df.loc[:, raw_test_df.columns.duplicated()].shape
# Check for missing values 
raw_test_df.isna().sum()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


<class 'pandas.core.frame.DataFrame'>
Index: 730 entries, 2190 to 2919
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            730 non-null    int64  
 1   pressure       730 non-null    float64
 2   maxtemp        730 non-null    float64
 3   temparature    730 non-null    float64
 4   mintemp        730 non-null    float64
 5   dewpoint       730 non-null    float64
 6   humidity       730 non-null    float64
 7   cloud          730 non-null    float64
 8   sunshine       730 non-null    float64
 9   winddirection  729 non-null    float64
 10  windspeed      730 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 68.4 KB


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,729.000000,730.000000
mean,183.000000,1013.503014,26.372466,23.963288,22.110274,20.460137,82.669863,76.360274,3.664384,103.923182,22.484247
std,105.438271,5.505871,5.672521,5.278098,5.170744,5.391169,7.818714,17.934121,3.639272,81.695458,9.954779
min,1.000000,1000.000000,7.400000,5.900000,4.200000,-0.000000,39.000000,0.000000,0.000000,10.000000,4.500000
25%,92.000000,1008.725000,21.600000,19.825000,17.825000,16.800000,77.250000,69.000000,0.325000,40.000000,14.500000
50%,183.000000,1012.700000,27.800000,25.650000,23.900000,22.300000,82.000000,83.000000,2.200000,70.000000,21.300000
75%,274.000000,1017.600000,31.000000,28.375000,26.400000,25.000000,89.000000,88.000000,6.675000,200.000000,28.400000
max,365.000000,1032.200000,35.800000,31.800000,29.100000,26.700000,98.000000,100.000000,11.800000,300.000000,59.500000


(730, 0)

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    1
windspeed        0
dtype: int64

In [200]:
test_df = raw_test_df.copy()
test_df.index.is_unique
# Remove leading and trailing whitespace form the column names
test_df.rename(columns=lambda x: x.strip(), inplace=True)
test_df.head()
test_df.tail()
test_df.shape
test_df.dropna(how='all').dropna(how='all', axis=1).shape
test_df.duplicated().any()
test_df.columns.duplicated().any()
for col in test_df:
    # The column is invariable if this is true
    if test_df[col].nunique() == 1:
        col

True

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2915,361,1020.8,18.2,17.6,16.1,13.7,96.0,95.0,0.0,20.0,34.3
2916,362,1011.7,23.2,18.1,16.0,16.0,78.0,80.0,1.6,40.0,25.2
2917,363,1022.7,21.0,18.5,17.0,15.5,92.0,96.0,0.0,50.0,21.9
2918,364,1014.4,21.0,20.0,19.7,19.8,94.0,93.0,0.0,50.0,39.5
2919,365,1020.9,22.2,18.8,17.0,13.3,79.0,89.0,0.2,60.0,50.6


(730, 11)

(730, 11)

np.False_

np.False_

In [201]:
test_nan_indexes = test_df.loc[test_df['winddirection'].isna()].index.tolist()
window_distance = 2
for i in test_nan_indexes:
    window = [j for j in range(i-window_distance, i+window_distance+1)]
    test_df.reindex(window)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2705,151,1008.4,34.4,30.0,27.8,24.8,72.0,22.0,11.1,230.0,25.2
2706,152,1007.6,33.1,30.4,28.9,25.9,81.0,87.0,1.6,100.0,35.3
2707,153,1007.8,32.9,30.6,28.9,22.0,65.0,75.0,8.2,NaN,17.2
2708,154,1008.6,33.7,30.5,28.2,25.0,89.0,83.0,3.9,220.0,11.7
2709,155,1009.5,33.9,30.2,27.5,25.8,74.0,30.0,10.8,220.0,15.8


## Sample submission

### Basic cleanup
#### The same data cleaning done to the train data will be done to the sample submission data

In [202]:
sample_submission_df = raw_sample_submission_df.copy()
sample_submission_df.index.is_unique
# Remove leading and trailing whitespace form the column names
sample_submission_df.rename(columns=lambda x: x.strip(), inplace=True)
sample_submission_df.head()
sample_submission_df.isna().sum()

True

,rainfall
id,
2190,0
2191,0
2192,0
2193,0
2194,0


rainfall    0
dtype: int64

In [203]:
train_df.to_csv(os.path.join(processed_dir, 'train.csv'))
test_df.to_csv(os.path.join(processed_dir, 'test.csv'))
sample_submission_df.to_csv(os.path.join(processed_dir, 'sample_submission.csv'))

### Feature engineering thoughts for machine learning pipelines
 * Derived and engineered features will be generated in the machine learning pipelines.
    * Simple derived features, such as 'expected_day', 'elapsed_days', and 'year' will be generated bu passing simple functions to apply.
    * More complicated engineered features, like 'season', will also be generated in the pipeline using something like K-means clustering.
 * Experiment with univariate, multivariate and/or KNN imputation for missing values.
    * There is only one missing value in the test data, so the method used for imputation will probably be inconsequential.
 * Experiment with standard scaling and min-max scaling for linear models.
 * Handle as much feature encoding as possible within the SciKit machine learning pipelines:
     * Nominal feature: OneHotEncoder
     * Ordinal features: OrdinalEncoder